# Image recognition with transfer learning


Computer vision has improved rapidly over the past decade due to a few intersecting trends:

 - Research advancements in machine learning
 - Petabytes of image data available online
 - Open Source culture with in AI community


One tool in a data scientists toolbox is **transfer learning**. This is particularly useful for image recognition, which typically requires a large amount of training data and extensive computation.


Instead of training a model from scratch, we can import the weights from another model a basis for our specific use case.



## InceptionV3

![](https://miro.medium.com/max/2000/0*te3xksOrVytM-B17)

InceptionV3 is one of the most advanced models for computer vision currently available. Created by Google Research members Szegedy et. al. The graphic above shows the complex architechture of this *very* deep network.

A link to the paper can be found here: https://arxiv.org/pdf/1512.00567.pdf




..


### Transfer Learning with InceptionV3

We can take advantage of this model, which has already been trained on thousands of images, and modify it for a specific use case.

My mother is an avid birdwatcher, and maybe she'd like to see a bird out in the wild, snap a photo, and see what kind of bird it is.

Using the Caltech Birds dataset from 2011, we can train an inception-based model to predict the species of bird pictured in a photograph




#### Read in data from tensorflow datasets

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

# load Caltech Birds dataset from 2011
(ds_train, ds_test), ds_info = tfds.load(
    'caltech_birds2011',
    split = ['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True)

print("Dataset info:\n\n", ds_info.description)

Dataset info:

 Caltech-UCSD Birds 200 (CUB-200) is an image dataset with photos 
of 200 bird species (mostly North American). The total number of 
categories of birds is 200 and there are 6033 images in the 2010 
dataset and 11,788 images in the 2011 dataset.
Annotations include bounding boxes, segmentation labels.



#### Data preprocessing
- normalize each image (256-bit color to float $\in$ \[0,1\] )
- pad and rescale (shrink/enlarge dimensiona to 400px, adding white space when nesessary)
- batch images (number of images to be held in memory for each epoch )
- prefetch (during training step s, allow the data for training step s+1 to be loaded to optimize runtime )

In [0]:
# Set batch size and image dimensions allowed by your memory resources
batch_size = 64
image_height = 400
image_width = 400


def norm(image, label):
    """Returns normalized image"""
    return tf.cast(image, tf.float32) / 255., label

def pad(image, label):
    """Returns resized image, padded where necessary"""
    return tf.image.resize_with_pad(image, image_height, image_width), label


# Prepare training data 
ds_train = ds_train.map(norm, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(pad, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
# Prepare validation data
ds_test = ds_test.map(norm, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(pad, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


#### Load in InceptionV3 model for transfer learning

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


pretrained_model = InceptionV3(input_shape=(image_height,image_width,3),
                               include_top=False,
                               weights='imagenet')

# Iterate through layers and make untrainable
for layer in pretrained_model.layers:
    layer.trainable = False

# Flatten the output, and add fully connected layer with a node for each class
x = tf.keras.layers.Flatten()(pretrained_model.output)
x = tf.keras.layers.Dense(200, activation='softmax')(x)

# Use our model as the input layer
model = tf.keras.Model(pretrained_model.input, x)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

"""Uncomment the last line to see the model summary, warning: it is very long""" 
#model.summary()

'Uncomment the last line to see the model summary, warning: it is very long'

# Model Training

We being by importing the pretrained model and setting the input shape to match the images we seek to classify

In [0]:
# Fit the model
history = model.fit(
    ds_train,
    epochs=50,
    shuffle=False,
    batch_size=batch_size
)

Epoch 1/50
94/94 [==============================] - 150s 2s/step - loss: 3.9861 - accuracy: 0.8105
Epoch 2/50
94/94 [==============================] - 150s 2s/step - loss: 2.0597 - accuracy: 0.8857
Epoch 3/50
94/94 [==============================] - 147s 2s/step - loss: 1.4462 - accuracy: 0.9117
Epoch 4/50
94/94 [==============================] - 147s 2s/step - loss: 1.2344 - accuracy: 0.9289
Epoch 5/50
94/94 [==============================] - 155s 2s/step - loss: 1.2834 - accuracy: 0.9331
Epoch 6/50
94/94 [==============================] - 153s 2s/step - loss: 1.2593 - accuracy: 0.9369
Epoch 7/50
94/94 [==============================] - 152s 2s/step - loss: 1.4441 - accuracy: 0.9298
Epoch 8/50
94/94 [==============================] - 155s 2s/step - loss: 1.3298 - accuracy: 0.9436
Epoch 9/50
94/94 [==============================] - 153s 2s/step - loss: 1.3134 - accuracy: 0.9441
Epoch 10/50
94/94 [==============================] - 153s 2s/step - loss: 1.1154 - accuracy: 0.9535
Epoch 11/